# Homework 4

Eugenio Baldo, Giovanni Giunta and  Leonardo Masci - Group 22

## Exercise 1

The goal of this first exercise was to gain an understanding of hashing and the HyperLogLog algorithm, the state of the art when it comes to estimating the number of unique users.
In order to accomplish this task we have consulted a number of resources, including the following paper ( http://algo.inria.fr/flajolet/Publications/FlFuGaMe07.pdf ) from which we got a lot of the formulas we then implemented in our HLL function.

Our work for this exercise is articulated in three steps:
1. Transform the given dataset into a set of binaries
2. Implement our HLL function
3. Estimate the number of unique elements

### Creating the binary file

First of all, we import everything we need and visualise the given initial txt file.

In [1]:
import pandas as pd
import itertools
import json
import math

In [2]:
dataset = pd.read_csv('hash.txt', header=None)

dataset.head()

,0
0,844082e02a27ddee8d99ea1af94a2969
1,ff96d6665b5c59d3a70bb8f2ba4f10be
2,b64a85884e2b159829331c19e05dbac9
3,1c8836719e84867c26ba2cfeb372c53d
4,b66f73ffd9008d9c99159e164261df51


The dataset is comprised of hexadecimal numbers. Therefore, in order to create our binary file we simply had to convert those numbers into binaries. A problem we encountered with this step was that not all the output values were of the same length, which created problems in the following steps. For this reason, we decided to add 0s as needed to reach the desired length for each binary value. We included the results in a txt file to avoid having the recreate this dataset every time.

# Possiamo rimuovere il print, no?

In [63]:
f = open("binaries.txt", "w")

for i, row in dataset.iterrows():            
    bin_value = bin(int(row[0], 16))[2:].zfill(len(row[0]) * 4)
    f.write(bin_value + '\n')

    if (i % 10000000) == 0:
        print(i)

f.close()

In [3]:
binaries = pd.read_csv('binaries.txt', header=None)

binaries.head()

,0
0,1000010001000000100000101110000000101010001001...
1,1111111110010110110101100110011001011011010111...
2,1011011001001010100001011000100001001110001010...
3,0001110010001000001101100111000110011110100001...
4,1011011001101111011100111111111111011001000000...


### Implementing our HyperLogLog function

Given the size of the binaries and the amount of values we were working with, we decided that 16 would be a good number of bytes to determine the destination bucket of each value, as we would end up with 2^16 buckets. We also initialised a dictionary that had 0 as the value for every single bucket.

We then counted the number of 0s found in each string binary value until the first 1 was encountered. Lastly, we confronted this number with the number already present for the corresponding bucket: if the new value was higher we substituted it. This way we ended up with a dictionary containing the highest amount of consecutive 0s for each bucket.

# togliere il print?

In [57]:
b = 16
values = [int("".join(map(str, list(i))), 2) for i in itertools.product([0, 1], repeat=b)]
db = {x: 0 for x in values}

def hyperLogLog(binary, b):
    bucket = int(binary[:b], 2)
    rest = binary[b:]
    count = 0
    for char in rest:
        if char == '0':
            count += 1
        else:
            count += 1
            break

    if db[bucket] < count:
        db[bucket] = count

    if (i % 10000000) == 0:
        print(i)

Once the function was ready, we applied it to our binary dataset and saved the resulting dictionary as a json file, again for easy future access.

In [4]:
for i, row in binaries.iterrows():
    binary = row[0]
    hyperLogLog(binary, b)


In [90]:
with open("db.json", "w") as outfile: 
    json.dump(db, outfile, indent = 4)

In [5]:
with open('db.json') as f:
    dt = json.load(f)

### Estimating amount of unique elements

Using the formulas we found in the above mentioned paper, we calculated the following values based on our final dictionary.

In [6]:
def calc_z(values):
    z = 0
    for x in values:
        z += 2 ** (-x)
    return z

m = len(dt)
values = list(dt.values())
z = calc_z(values)
alpha = 0.7213/(1 + 1.079 / m )
E = round((alpha * float(m**2)) / z)
error = 1.04 / (math.sqrt(m))

In [7]:
E

125674524

In [8]:
error

0.0040625

In conclusion, we estimated that the cardinality of our initial dataset was of about 125 million, with an approximated error of 0.004.